In [ ]:
!pip install langchain
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "발급받은 API 키"

## LangChain을 이용한 기본적인 챗봇

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("""
        다음은 인간과 AI의 친근한 대화입니다.
        AI는 말이 많고, 맥락에 따라 구체적인 내용을 많이 알려줍니다.
        AI는 질문에 대한 답을 모르면 정직하게 "모른다"고 대답합니다.
    """),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

while True:
    command = input(">")
    if command == "q":
        break
    response = conversation.predict(input=command)
    print(f"AI:{response}")

In [ ]:
print(memory)

## 데이터베이스 생성 (7.3절 참조)

In [ ]:
import sqlite3

# 데이터베이스 접속 생성
conn = sqlite3.connect('user_support.db')

# 커서 객체 생성
cursor = conn.cursor()

# users 테이블 생성
cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT
)
''')

# products 테이블 생성
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price INTEGER
)
''')

# order_history 테이블 생성
cursor.execute('''
CREATE TABLE order_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    purchase_date DATE,
    quantity INTEGER,
    remarks TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

# support_history 테이블 생성
cursor.execute('''
CREATE TABLE support_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    subject TEXT,
    message_content TEXT,
    message_type TEXT,
    status TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
)
''')

# 변경 사항 커밋
conn.commit()

# 데이터베이스 접속 종료
conn.close()

In [ ]:
import sqlite3
from datetime import datetime

# 데이터베이스 접속 생성
conn = sqlite3.connect('user_support.db')

# 커서 객체 생성
cursor = conn.cursor()

# users 테이블에 한국어 테스트 데이터 삽입
users_data = [
    (1, '태진', '윤', 'taejin@example.com', '099-1234-5678'),
    (2, '은미', '김', 'eunmi@example.com', '098-9876-5432')
]

for user in users_data:
    cursor.execute('''
    INSERT INTO users (user_id, first_name, last_name, email, phone)
    VALUES (?, ?, ?, ?, ?)
    ''', user)

# products 테이블에 한국어 테스트 데이터 삽입
products_data = [
    (1, '제품A', 10000),
    (2, '제품B', 20000),
    (3, '제품C', 30000)
]

for product in products_data:
    cursor.execute('''
    INSERT INTO products (product_id, product_name, price)
    VALUES (?, ?, ?)
    ''', product)

# order_history 테이블에 한국어 테스트 데이터 삽입
order_history_data = [
    (1, 1, 1, '2023-10-01', 2, '빠른 발송', datetime.now()),
    (2, 2, 3, '2023-10-05', 1, '선물 포장', datetime.now())
]

for order in order_history_data:
    cursor.execute('''
    INSERT INTO order_history (history_id, user_id, product_id, purchase_date, quantity, remarks, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', order)

# support_history 테이블에 한국어 테스트 데이터 삽입
support_history_data = [
    (1, 1, '청구 관련 문제', None, None, 'open', datetime.now()),
    (2, 1, None, '청구에 문제가 있습니다.', 'user', None, datetime.now()),
    (3, 1, None, '문의해주셔서 감사합니다. 문제를 조사하고 있습니다.', 'support', None, datetime.now())
]

for support in support_history_data:
    cursor.execute('''
    INSERT INTO support_history (history_id, user_id, subject, message_content, message_type, status, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', support)

# 변경 사항 커밋
conn.commit()

# 데이터베이스 접속 종료
conn.close()

## SQL 질의 방법

In [ ]:
!pip install langchain-experimental

In [ ]:
from langchain import OpenAI
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate

sql_uri = "sqlite:///user_support.db"
db = SQLDatabase.from_uri(sql_uri)
llm = ChatOpenAI(temperature=0.2)
template = \
    """
    사용자 테이블만 대상으로 합니다.
    답변은 JSON 형식으로 표시해주세요: [{question}]
    예:
    [{{
        "last_mame": "",
        "first_name": "",
        "phone": "",
        "email": ""
    }}]
    """
prompt = PromptTemplate(template=template, input_variables=["question"])
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, output_key="Answer")
user = db_chain.run(prompt.format(question="모든 사용자를 알려주세요"))
print(user)

## 챗봇 구현

### ① 사용자 식별

In [ ]:
import json

sql_uri = "sqlite:///user_support.db"
db = SQLDatabase.from_uri(sql_uri)
llm = ChatOpenAI(temperature=0.2)

def find_user(user_text):
    template = \
        """
        사용자 테이블만 대상으로 합니다. 다음 요청문에 대해 사용자를 고유하게 식별하고 싶습니다.
        답변은 예와 같이 JSON 형식으로 표시해주세요: 요청문[{question}]

        예:
        [{{
            "user_id": ,
            "last_mame": "",
            "first_name": "",
            "phone": "",
            "email": ""
        }}]
    """
    prompt = PromptTemplate(template=template, input_variables=["question"])
    db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, output_key="Answer")
    return db_chain.run(prompt.format(question=user_text))

if __name__ == "__main__":
    print("\n고객 정보를 확인합니다. 이름, 전화번호, 사용자 ID, 이메일 등 고객을 식별할 수 있는 정보를 입력해주세요.")
    while True:
        user_text = input(">")
        user_json = find_user(user_text)
        try:
            users = json.loads(user_json)
        except:
            print("고객 정보를 확인할 수 없습니다. 다시 입력해주세요.")
            continue

        if users is None:
            print("고객 정보를 확인할 수 없습니다. 다시 입력해주세요.")
            continue
        if len(users) > 1:
            print("고객의 고유 정보를 식별할 수 없습니다. 다시 입력해주세요.")
            continue

        print("고객 정보를 확인했습니다. 감사합니다.")
        break

    user = '\n'.join(f'{key}: {value}' for key, value in users[0].items())

    print(user)

### ② 해당 사용자에 대한 그 외의 정보를 데이터베이스에서 획득

In [ ]:
from langchain.chains import SQLDatabaseSequentialChain

def get_user_info(user):
    template = \
        """
        다음 요청문에서 지정된 사용자 ID의 제품 이름, 가격도 포함한 구매 이력(테이블) 내용을 알고 싶습니다.
        답변은 헤더가 있는 CSV 형식으로 출력해 주시기 바랍니다.
        요청문[{user}]
        """
    prompt = PromptTemplate(template=template, input_variables=["user"])
    chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)
    order_history = chain.run(prompt.format(user=user))

    template = \
        """
        다음 요청문에서 지정된 사용자 ID의 대응 이력(테이블) 내용을 알고 싶습니다.
        답변은 헤더가 있는 CSV 형식으로 출력해주시기 바랍니다.
        요청문[{user}]
        """
    prompt = PromptTemplate(template=template, input_variables=["user"])
    chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)
    support_history = chain.run(prompt.format(user=user))

    return f"구매 이력:\n{order_history}\n\n대응 이력:\n{support_history}\n\n"

if __name__ == "__main__":
    user_info = get_user_info(user)

In [ ]:
import pprint

pprint.pprint(user_info)

### ③ 지원 실시

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

template = \
    """
    ## 사용자 정보
        {user}
        {user_info}
    ## 처리
    당신은 고객 응대 전문가입니다.고객의 요구사항에 대해 정확하게 답변해주세요.
    고객 응대 시, 위의 사용자 정보를 참고하여 친절하고 정중하게 응대해주세요.
    단, 모르는 것은 모른다고 대답해주세요.
    """

system_template = template.format(user=user, user_info=user_info)
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0.5)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

while True:
    command = input("질문을 기다리고 있습니다 (q로 종료)>")

    if command == "q":
        break

    response = conversation.predict(input=command)
    print(f"{response}\n")